In [ ]:

"""

ля 151 406 договоров страхования транспортных средств известны значения ряда признаков, 
в том числе пол, возраст, стаж вождения и коэффициент бонус-малус водителя, тип, марка, модель, 
год выпуска, страна – производитель, мощность и объем двигателя, а также признак target, равный 1, 
если заключение договора с клиентом является рисковым, и 0 в противном случае (файл insclass_train.csv).

В обучающем наборе данных для каждого договора известны следующие поля:

variable_1 - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);
variable_2 - индикатор расторжения договора по инициативе страхователя (клиента);
variable_3 - индикатор расторжения договора по инициативе страховщика (страховой компании);
variable_4 - идентификатор года выпуска транспортного средства;
variable_5 - идентификатор страны - производителя транспортного средства;
variable_6 - мощность двигателя в лошадиных силах;
variable_7 - объем двигателя в куб. см;
variable_8 - идентификатор стороны расположения руля (левый или правый);
variable_9 - пробег транспортного средства, покрываемый гарантией производителя;
variable_10 - индикатор действия гарантии на транспортное средство;
variable_11 - "мультидрайв" - индикатор допуска к управлению транспортным средством более одного водителя;
variable_12 - возраст транспортного средства (в мес.);
variable_13 - возраст водителя с максимальным стажем;
variable_14 - коэффициент возраст-стаж;
variable_15 - коэффициент краткосрочности;
variable_16 - коэффициент мощности;
variable_17 - коэффициент "мультидрайв";
variable_18 - территориальный коэффициент;
variable_19 - коэффициент "КНДР";
variable_20 - идентификатор канала продаж;
variable_21 - марка транспортного средства;
variable_22 - модель транспортного средства;
variable_23 - индикатор отечественных транспортных средств;
variable_24 - пол водителя с максимальным коэффициентом "возраст-стаж";
variable_25 - индикатор пролонгации;
variable_26 - индикатор совпадения собственника транспортного средства и водителя;
variable_27 - стаж водителя с максимальным коэффициентом "возраст-стаж";
variable_28 - тип транспортного средства;
target - класс риска, равный 1, если заключение договора с клиентом является рисковым, и 0 в противном случае.
В тестовом наборе данных каждому договору соответствует уникальный идентификатор id.
"""

'\nВ обучающем наборе данных для каждого договора известны следующие поля:\n\nvariable_1 - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);\nvariable_2 - индикатор расторжения договора по инициативе страхователя (клиента);\nvariable_3 - индикатор расторжения договора по инициативе страховщика (страховой компании);\nvariable_4 - идентификатор года выпуска транспортного средства;\nvariable_5 - идентификатор страны - производителя транспортного средства;\nvariable_6 - мощность двигателя в лошадиных силах;\nvariable_7 - объем двигателя в куб. см;\nvariable_8 - идентификатор стороны расположения руля (левый или правый);\nvariable_9 - пробег транспортного средства, покрываемый гарантией производителя;\nvariable_10 - индикатор действия гарантии на транспортное средство;\nvariable_11 - "мультидрайв" - индикатор допуска к управлению транспортным средством более одного водителя;\nvariable_12 - возраст транспорт

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier

TRAIN_PATH = "insclass_train.csv"
TEST_PATH  = "insclass_test.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

y = train["target"].astype(int).values
X = train.drop(columns=["target"]).copy()

test_id = test["id"].values
X_test = test.drop(columns=["id"]).copy()

drop_cols = []
for c in X.columns:
    # константа в train
    if X[c].nunique(dropna=False) <= 1:
        drop_cols.append(c)
if X_test["variable_15"].isna().mean() > 0.999:
    drop_cols.append("variable_15")


drop_cols = sorted(set(drop_cols))
X.drop(columns=drop_cols, inplace=True, errors="ignore")
X_test.drop(columns=drop_cols, inplace=True, errors="ignore")

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype("string").fillna("__MISSING__")
    X_test[c] = X_test[c].astype("string").fillna("__MISSING__")

na_cols = [c for c in X.columns if X[c].isna().any()]
for c in na_cols:
    X[c + "__isna"] = X[c].isna().astype(np.uint8)
    X_test[c + "__isna"] = X_test[c].isna().astype(np.uint8)

cat_cols2 = [c for c in X.columns if X[c].dtype == "string" or X[c].dtype == "object"]
cat_idx = [X.columns.get_loc(c) for c in cat_cols2]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(X), dtype=float)
test_pred = np.zeros(len(X_test), dtype=float)

params = dict(
    loss_function="Logloss",
    eval_metric="AUC",
    iterations=8000,
    learning_rate=0.03,
    depth=8,
    l2_leaf_reg=20,
    random_strength=1.0,
    border_count=128,
    auto_class_weights="Balanced",
    random_seed=42,
    verbose=200
)

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, y_tr = X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = X.iloc[va_idx], y[va_idx]

    model = CatBoostClassifier(**params)
    model.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        cat_features=cat_idx,
        use_best_model=True,
        early_stopping_rounds=400
    )

    oof[va_idx] = model.predict_proba(X_va)[:, 1]
    test_pred += model.predict_proba(X_test)[:, 1] / skf.n_splits


thr_grid = np.linspace(0.01, 0.99, 199)
best_thr, best_score = None, -1

for t in thr_grid:
    pred_bin = (oof >= t).astype(int)
    s = roc_auc_score(y, pred_bin)  # именно для бинарного предикта
    if s > best_score:
        best_score = s
        best_thr = t

print("OOF AUC(on 0/1) =", best_score, "best_thr =", best_thr)

test_bin = (test_pred >= best_thr).astype(int)

sub = pd.DataFrame({"id": test_id, "target": test_bin})
sub.to_csv("submission.csv", index=False)
print(sub.head())


Train rows: 151406 Test rows: 22624
  variable_1  variable_2  variable_3  variable_4 variable_5  variable_6  \
0       w200           0           0          14         q2        98.0   
1       w160           0           0           7        q11       106.0   

   variable_7  variable_8  variable_9  variable_10  ...  variable_20  \
0         NaN         0.0         NaN            0  ...            C   
1         NaN         0.0         NaN            0  ...            C   

   variable_21  variable_22  variable_23  variable_24  variable_25  \
0           j2          h45            0          0.0            0   
1          j33         h234            0          1.0            0   

   variable_26  variable_27  variable_28 target  
0            1    19.323463           t1      0  
1            1    41.177900           t1      0  

[2 rows x 29 columns]
   id variable_1  variable_2  variable_3  variable_4 variable_5  variable_6  \
0   1       w200           0           0          12      